In [1]:
# Linalg modules
import numpy as np

# Plotting modules
import matplotlib.pyplot as plt

# ASE modules
import ase
import pickle
from tqdm import tqdm
from ase.calculators.cp2k import CP2K

In [2]:
frame=0
with open("/tikhome/mgern/Desktop/AdvancedSimMethods/ml-module/configuration-space-exploration/selected_configurations.pkl",'rb') as f:
     # Load one data frame here
    data = pickle.load(f)
    frame=data[0]

2024-05-02 14:56:54.592722: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-05-02 14:56:54.592837: E external/xla/xla/stream_executor/cuda/cuda_diagnostics.cc:256] kernel version 535.171.4 does not match DSO version 550.67.0 -- cannot find working devices in this configuration
CUDA backend failed to initialize: FAILED_PRECONDITION: No visible GPU devices. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [27]:
def build_cp2k_iniut(cutoff: int) -> str:
    """
    Build the CP2K input file for varying cutoff.

    Parameters
    ----------
    cutoff : int
        The cutoff energy in Rydberg.
    """
    return f"""
        &FORCE_EVAL
        &DFT
        # &MGRID
        #             CUTOFF {cutoff}
        # &END MGRID
        # &VDW_POTENTIAL
        #     POTENTIAL_TYPE PAIR_POTENTIAL
        #     &PAIR_POTENTIAL
        #         PARAMETER_FILE_NAME /group/allatom/cp2kv2024.1/data/vdW_kernel_table.dat
        #         REFERENCE_FUNCTIONAL PBE
        #         TYPE DFTD3
        #     &END PAIR_POTENTIAL
        # &END VDW_POTENTIAL
        &SCF
                    SCF_GUESS RESTART
                    IGNORE_CONVERGENCE_FAILURE
        &END SCF
        &END DFT
        &END FORCE_EVAL
    """

In [29]:
# This is required for ASE to work with CP2K. If you are working from your own computer, you may need to change this path.
CP2K.command = "/group/allatom/cp2kv2024.1/exe/local/cp2k_shell.psmp"

cutoffs = np.linspace(100,500,10)  # Fill this in

energies = []
forces = []

for cutoff in tqdm(cutoffs):
    cp2k_input = build_cp2k_iniut(cutoff)

    calculator = CP2K(xc="BEEF", inp=cp2k_input)  # Adapt this functional for your system.

    # Create the atoms object
    atoms = ase.Atoms("Ar64", positions=frame, pbc=[1, 1, 1], cell=np.ones(3) * 14.474693)
    energies.append(calculator.get_potential_energy(atoms))
    forces.append(calculator.get_forces(atoms))

  0%|          | 0/10 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Plot energy and force as a function of cutoff
np.savetxt("energies_pbe.txt", energies)
np.savetxt("forces_pbe.txt", forces)
fig, ax = plt.subplots(2, 1, figsize=(6, 8))
ax[0].plot(cutoffs, energies, "o-")
ax[0].set_xlabel("Cutoff (Rydberg)")
ax[0].set_ylabel("Energy (eV)")
ax[1].plot(cutoffs, forces, "o-")
ax[1].set_xlabel("Cutoff (Rydberg)")
ax[1].set_ylabel("Force (eV/Angstrom)")
plt.tight_layout()
plt.savefig("convergence.pdf")
plt.show()
